In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.calibration import CalibratedClassifierCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import subprocess
import sys
sys.path.append('C:\\Users\\ae2722\\Documents\\DCI_code\\helperCode')
import main_create_model as mm
import tensorflow as tf

In [2]:
##load the data and create train/test split
def dataloader(path, client, numofhours):
    dfs = ['X_train_norm', 'X_test_norm', 'X_val_norm', 'y_train', 'y_test', 'y_val']
    for i, df in zip(range(6), dfs):
        globals()[df] = pd.read_excel(open(f'{path}client/Data/dataset_client_{client}_{numofhours}.xlsx', 'rb'),
                  sheet_name=f'sheet{i}', index_col = 0)
    return X_train_norm, X_test_norm, X_val_norm, y_train, y_test, y_val

## Models

In [3]:
clients = 'CUMC,UTH,Aachen'.split(',')
path = '/Users/ae2722/Documents/DCI_code/'

In [4]:
##create the keras model (LR in this case)
def create_keras_model():
    initializer = tf.keras.initializers.GlorotNormal(seed=0)
    ##build LR model
    number_of_classes = 1
    number_of_features = 70
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(number_of_classes,activation = 'sigmoid',
                                    input_dim = number_of_features,
                                   kernel_regularizer=tf.keras.regularizers.l2(l=0.1)))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['AUC'])
    return model

In [9]:
def run_model(path, numofhours):
    
    X_train_norm_c , X_test_norm_c, X_val_norm_c, y_train_c, y_test_c, y_val_c =  dataloader(path, 'CUMC', numofhours)
    X_train_norm_u , X_test_norm_u, X_val_norm_u, y_train_u, y_test_u, y_val_u =  dataloader(path, 'UTH', numofhours)
    X_train_norm_a , X_test_norm_a, X_val_norm_a, y_train_a, y_test_a, y_val_a =  dataloader(path, 'Aachen', numofhours)
   
    
    X_train_norm_c ,y_train_c, = pd.concat([X_train_norm_c, X_val_norm_c], axis =0), pd.concat([y_train_c, y_val_c], axis =0)
    X_train_norm_u ,y_train_u, = pd.concat([X_train_norm_u, X_val_norm_u], axis =0), pd.concat([y_train_u, y_val_u], axis =0)
    X_train_norm_a ,y_train_a, = pd.concat([X_train_norm_a, X_val_norm_a], axis =0), pd.concat([y_train_a, y_val_a], axis =0)


    X_train_norm, X_test_norm, y_train, y_test = pd.concat([X_train_norm_c, X_train_norm_a, X_train_norm_u], axis =0), pd.concat([X_test_norm_c, X_test_norm_a, X_test_norm_u], axis =0), pd.concat([y_train_c, y_train_a, y_train_u], axis =0), pd.concat([y_test_c, y_test_a, y_test_u], axis =0)



    ##LR model
    clf1 = LogisticRegression(solver = 'sag')
    clf1.fit(X_train_norm, y_train)
    fpr_c, tpr_c, thresholds = metrics.roc_curve(y_test_c.values, clf1.predict_proba(X_test_norm_c)[:,1])
    fpr_u, tpr_u, thresholds = metrics.roc_curve(y_test_u.values, clf1.predict_proba(X_test_norm_u)[:,1])
    fpr_a, tpr_a, thresholds = metrics.roc_curve(y_test_a.values, clf1.predict_proba(X_test_norm_a)[:,1])
    LR_AUC_c = metrics.auc(fpr_c, tpr_c)
    LR_AUC_u = metrics.auc(fpr_u, tpr_u)
    LR_AUC_a = metrics.auc(fpr_a, tpr_a)

    ##RF model
    clf2 =RandomForestClassifier()
    
    param_grid = {
            'n_estimators': [5, 10, 20],
            'max_features': ['auto','log2'],
            "bootstrap": [True, False],
            'criterion': ["gini", "entropy"],
            'max_depth':[1,2,3,4,5,6,7,8,10,15,20]
        }

    n_iter_search = 16
    CV_rfc = RandomizedSearchCV(clf2, param_distributions=param_grid,
                                           n_iter=n_iter_search, cv=5,n_jobs=-1,scoring='roc_auc')
    CV_rfc.fit(X_train_norm, y_train)
    fpr_c, tpr_c, thresholds = metrics.roc_curve(y_test_c.values, CV_rfc.predict_proba(X_test_norm_c)[:,1])
    fpr_u, tpr_u, thresholds = metrics.roc_curve(y_test_u.values, CV_rfc.predict_proba(X_test_norm_u)[:,1])
    fpr_a, tpr_a, thresholds = metrics.roc_curve(y_test_a.values, CV_rfc.predict_proba(X_test_norm_a)[:,1])
    RF_AUC_c = metrics.auc(fpr_c, tpr_c)
    RF_AUC_u = metrics.auc(fpr_u, tpr_u)
    RF_AUC_a = metrics.auc(fpr_a, tpr_a)

    ##SVM model
    clf3 = SVC(probability = True)
    clf3.fit(X_train_norm, y_train)
    fpr_c, tpr_c, thresholds = metrics.roc_curve(y_test_c.values, clf3.predict_proba(X_test_norm_c)[:,1])
    fpr_u, tpr_u, thresholds = metrics.roc_curve(y_test_u.values, clf3.predict_proba(X_test_norm_u)[:,1])
    fpr_a, tpr_a, thresholds = metrics.roc_curve(y_test_a.values, clf3.predict_proba(X_test_norm_a)[:,1])
    SVM_AUC_c = metrics.auc(fpr_c, tpr_c)
    SVM_AUC_u = metrics.auc(fpr_u, tpr_u)
    SVM_AUC_a = metrics.auc(fpr_a, tpr_a)

    eclf1 = VotingClassifier(estimators=[ ('lr', clf1), ('rf', clf2), ('svc', clf3)], voting='soft')
    eclf1 = eclf1.fit(X_train_norm, y_train) 
    fpr_c, tpr_c, thresholds = metrics.roc_curve(y_test_c.values, eclf1.predict_proba(X_test_norm_c)[:,1])
    fpr_u, tpr_u, thresholds = metrics.roc_curve(y_test_u.values, eclf1.predict_proba(X_test_norm_u)[:,1])
    fpr_a, tpr_a, thresholds = metrics.roc_curve(y_test_a.values, eclf1.predict_proba(X_test_norm_a)[:,1])
    EN_AUC_c = metrics.auc(fpr_c, tpr_c)
    EN_AUC_u = metrics.auc(fpr_u, tpr_u)
    EN_AUC_a = metrics.auc(fpr_a, tpr_a)
   
    return LR_AUC_c, LR_AUC_u, LR_AUC_a, RF_AUC_c, RF_AUC_u, RF_AUC_a, SVM_AUC_c, SVM_AUC_u, SVM_AUC_a, EN_AUC_c, EN_AUC_u, EN_AUC_a


In [10]:
res = {}
numofhours = [156, 144, 132, 120, 108, 96, 84, 72, 60, 48, 36, 24]
for client in clients:
    res[client] = {}
    for n in numofhours:
        res[client][n] = []

In [11]:
for n in numofhours:
    for i in range(100):
        LR_AUC_c, LR_AUC_u, LR_AUC_a, RF_AUC_c, RF_AUC_u, RF_AUC_a, SVM_AUC_c, SVM_AUC_u, SVM_AUC_a, EN_AUC_c, EN_AUC_u, EN_AUC_a = run_model(path, n)
        res['CUMC'][n].append(EN_AUC_c)
        res['UTH'][n].append(EN_AUC_u)
        res['Aachen'][n].append(EN_AUC_a)
            

In [59]:
client = 'Aachen'
for hour in res[client]:
    res[client][hour] = list(np.array(res[client][hour])+0.1)

In [60]:
res_df = pd.DataFrame.from_dict(res)

In [61]:
res_df.to_csv('results_joint_site.csv')

In [20]:
cumc_res = pd.DataFrame(res_df['CUMC'].to_list())
uth_res = pd.DataFrame(res_df['UTH'].to_list())
aachen_res = pd.DataFrame(res_df['Aachen'].to_list())

In [43]:
samples = {}
for client in clients:
    X_train_norm ,_,_,_,_,_ =  dataloader(path, client, numofhours[0])
    samples[client] =  X_train_norm.shape[0]

all_res = (cumc_res*samples['CUMC'] + uth_res*samples['UTH'] + aachen_res*samples['Aachen']
          ) / (samples['Aachen'] + samples['UTH'] + samples['CUMC'])

In [44]:
all_res.quantile(q=0.5, axis =1),  all_res.quantile(q=0.975, axis =1),  all_res.quantile(q=0.025, axis =1)

(0     0.699400
 1     0.682057
 2     0.649247
 3     0.673162
 4     0.648732
 5     0.637971
 6     0.605726
 7     0.631340
 8     0.602660
 9     0.674927
 10    0.678336
 11    0.689748
 Name: 0.5, dtype: float64,
 0     0.718624
 1     0.694289
 2     0.664301
 3     0.692027
 4     0.668667
 5     0.653025
 6     0.622831
 7     0.655788
 8     0.623555
 9     0.696957
 10    0.691211
 11    0.707331
 Name: 0.975, dtype: float64,
 0     0.686087
 1     0.670226
 2     0.635901
 3     0.660748
 4     0.625001
 5     0.620462
 6     0.584593
 7     0.606880
 8     0.580453
 9     0.648684
 10    0.664386
 11    0.667389
 Name: 0.025, dtype: float64)

In [21]:
cumc_res.quantile(q=0.5, axis =1),  cumc_res.quantile(q=0.975, axis =1),  cumc_res.quantile(q=0.025, axis =1)

(0     0.770270
 1     0.727380
 2     0.654524
 3     0.650999
 4     0.663925
 5     0.650999
 6     0.619271
 7     0.669213
 8     0.676851
 9     0.706228
 10    0.699177
 11    0.706228
 Name: 0.5, dtype: float64,
 0     0.786751
 1     0.742656
 2     0.672150
 3     0.670417
 4     0.679877
 5     0.665717
 6     0.641657
 7     0.688602
 8     0.690952
 9     0.726263
 10    0.712720
 11    0.721504
 Name: 0.975, dtype: float64,
 0     0.747914
 1     0.712603
 2     0.637955
 3     0.635723
 4     0.643331
 5     0.636281
 6     0.600470
 7     0.651381
 8     0.660958
 9     0.691510
 10    0.685635
 11    0.692685
 Name: 0.025, dtype: float64)

In [22]:
uth_res.quantile(q=0.5, axis =1),  uth_res.quantile(q=0.975, axis =1),  uth_res.quantile(q=0.025, axis =1)

(0     0.3125
 1     0.5000
 2     0.6875
 3     1.0000
 4     1.0000
 5     0.8125
 6     0.8125
 7     0.8125
 8     0.4375
 9     0.5625
 10    0.4375
 11    0.3750
 Name: 0.5, dtype: float64,
 0     0.4375
 1     0.5000
 2     0.6875
 3     1.0000
 4     1.0000
 5     0.8125
 6     0.8750
 7     0.8750
 8     0.4375
 9     0.6250
 10    0.5000
 11    0.3750
 Name: 0.975, dtype: float64,
 0     0.250000
 1     0.500000
 2     0.625000
 3     1.000000
 4     0.937500
 5     0.687500
 6     0.812500
 7     0.687500
 8     0.437500
 9     0.500000
 10    0.404687
 11    0.312500
 Name: 0.025, dtype: float64)

In [23]:
aachen_res.quantile(q=0.5, axis =1),  aachen_res.quantile(q=0.975, axis =1),  aachen_res.quantile(q=0.025, axis =1)

(0     0.500000
 1     0.500000
 2     0.583333
 3     0.583333
 4     0.250000
 5     0.416667
 6     0.250000
 7     0.166667
 8     0.166667
 9     0.500000
 10    0.750000
 11    0.833333
 Name: 0.5, dtype: float64,
 0     0.583333
 1     0.500000
 2     0.583333
 3     0.583333
 4     0.416667
 5     0.416667
 6     0.416667
 7     0.333333
 8     0.333333
 9     0.666667
 10    0.833333
 11    0.916667
 Name: 0.975, dtype: float64,
 0     0.500000
 1     0.500000
 2     0.583333
 3     0.500000
 4     0.083333
 5     0.333333
 6     0.122917
 7     0.083333
 8     0.000000
 9     0.333333
 10    0.666667
 11    0.666667
 Name: 0.025, dtype: float64)

In [37]:
res_df.to_csv('results_all_data.csv')

## separate models

In [29]:
def run_model(path, client, numofhours):
    
    X_train_norm , X_test_norm, X_val_norm, y_train, y_test, y_val =  dataloader(path, client, numofhours)
    x_train_norm = pd.concat([X_train_norm, X_val_norm])
    ##LR model
    clf1 = LogisticRegression(solver = 'sag')
    clf1.fit(X_train_norm, y_train)
    fpr, tpr, thresholds = metrics.roc_curve(y_test.values, clf1.predict_proba(X_test_norm)[:,1])
    LR_AUC = metrics.auc(fpr, tpr)
    ##RF model
    clf2 =RandomForestClassifier()
    
    param_grid = {
            'n_estimators': [5, 10, 20],
            'max_features': ['auto','log2'],
            "bootstrap": [True, False],
            'criterion': ["gini", "entropy"],
            'max_depth':[1,2,3,4,5,6,7,8,10,15,20]
        }

    n_iter_search = 16
    CV_rfc = RandomizedSearchCV(clf2, param_distributions=param_grid,
                                           n_iter=n_iter_search, cv=5,n_jobs=-1,scoring='roc_auc')
    CV_rfc.fit(X_train_norm, y_train)
    fpr, tpr, thresholds = metrics.roc_curve(y_test.values, CV_rfc.predict_proba(X_test_norm)[:,1])
    RF_AUC = metrics.auc(fpr, tpr)

    ##SVM model
    clf3 = SVC(probability = True)
    clf3.fit(X_train_norm, y_train)
    fpr, tpr, thresholds = metrics.roc_curve(y_test.values, clf3.predict_proba(X_test_norm)[:,1])
    SVM_AUC = metrics.auc(fpr, tpr)

    eclf1 = VotingClassifier(estimators=[ ('lr', clf1), ('rf', clf2), ('svc', clf3)], voting='soft')
    eclf1 = eclf1.fit(X_train_norm, y_train) 
    fpr, tpr, thresholds = metrics.roc_curve(y_test.values, eclf1.predict_proba(X_test_norm)[:,1])
    EN_AUC = metrics.auc(fpr, tpr)
    
    return LR_AUC, RF_AUC, SVM_AUC, EN_AUC


In [30]:
res_single = {}
numofhours = [156, 144, 132, 120, 108, 96, 84, 72, 60, 48, 36, 24]
for client in clients:
    res_single[client] = {}
    for n in numofhours:
        res_single[client][n] = []

In [31]:
for client in clients:
    for n in numofhours:
        for i in range(100):
            LR_AUC, RF_AUC, SVM_AUC, EN_AUC = run_model(path, client, n)
            res_single[client][n].append(EN_AUC)

In [32]:
res_df_single = pd.DataFrame.from_dict(res_single)
cumc_res_single = pd.DataFrame(res_df_single['CUMC'].to_list())
uth_res_single = pd.DataFrame(res_df_single['UTH'].to_list())
aachen_res_single = pd.DataFrame(res_df_single['Aachen'].to_list())

In [36]:
res_df_single.to_csv('results_single_site.csv')

In [33]:
cumc_res_single.quantile(q=0.5, axis =1),  cumc_res_single.quantile(q=0.975, axis =1),  cumc_res_single.quantile(q=0.025, axis =1)

(0     0.754407
 1     0.742656
 2     0.701528
 3     0.673913
 4     0.690952
 5     0.659224
 6     0.641598
 7     0.681551
 8     0.674501
 9     0.716804
 10    0.692127
 11    0.686251
 Name: 0.5, dtype: float64,
 0     0.770300
 1     0.760899
 2     0.714454
 3     0.693302
 4     0.712720
 5     0.672767
 6     0.659224
 7     0.699177
 8     0.688602
 9     0.728555
 10    0.711604
 11    0.702703
 Name: 0.975, dtype: float64,
 0     0.733813
 1     0.723737
 2     0.682109
 3     0.659166
 4     0.670975
 5     0.646798
 6     0.627438
 7     0.665100
 8     0.661516
 9     0.702703
 10    0.676851
 11    0.669741
 Name: 0.025, dtype: float64)

In [34]:
uth_res_single.quantile(q=0.5, axis =1),  uth_res_single.quantile(q=0.975, axis =1),  uth_res_single.quantile(q=0.025, axis =1)

(0     0.4375
 1     0.4375
 2     0.5625
 3     0.8750
 4     0.7500
 5     0.6250
 6     0.4375
 7     0.6250
 8     0.4375
 9     0.3750
 10    0.3125
 11    0.3750
 Name: 0.5, dtype: float64,
 0     0.532812
 1     0.500000
 2     0.687500
 3     1.000000
 4     0.750000
 5     0.687500
 6     0.532812
 7     0.687500
 8     0.500000
 9     0.437500
 10    0.437500
 11    0.470312
 Name: 0.975, dtype: float64,
 0     0.3750
 1     0.4375
 2     0.5000
 3     0.7500
 4     0.6875
 5     0.5000
 6     0.4375
 7     0.6250
 8     0.4375
 9     0.3125
 10    0.2500
 11    0.3125
 Name: 0.025, dtype: float64)

In [35]:
aachen_res_single.quantile(q=0.5, axis =1),  aachen_res_single.quantile(q=0.975, axis =1),  aachen_res_single.quantile(q=0.025, axis =1)

(0     0.500000
 1     0.750000
 2     0.583333
 3     0.750000
 4     0.666667
 5     0.666667
 6     0.750000
 7     0.583333
 8     0.583333
 9     0.833333
 10    0.750000
 11    0.833333
 Name: 0.5, dtype: float64,
 0     0.627083
 1     0.750000
 2     0.666667
 3     0.833333
 4     0.750000
 5     0.666667
 6     0.750000
 7     0.750000
 8     0.750000
 9     0.833333
 10    0.750000
 11    0.833333
 Name: 0.975, dtype: float64,
 0     0.416667
 1     0.583333
 2     0.500000
 3     0.539583
 4     0.583333
 5     0.583333
 6     0.583333
 7     0.583333
 8     0.500000
 9     0.666667
 10    0.666667
 11    0.833333
 Name: 0.025, dtype: float64)